Bayesian interpretation of medical tests
-----------------------------------------

This notebooks explores several problems related to interpreting the results of medical tests.

Copyright 2016 Allen Downey

MIT License: http://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

from thinkbayes2 import Pmf

In [2]:
from random import random

def flip(p):
    return random() < p

In [3]:
def run_single_simulation(func, iters=1000000):
    pmf_t = Pmf([0.2, 0.4])
    p = 0.1
    s = 0.9

    outcomes = Pmf()
    for i in range(iters):
        test, sick = func(p, s, pmf_t)
        if test:
            outcomes[sick] += 1

    outcomes.Normalize()
    return outcomes

Scenario A:  Choose `t` for each patient, yield all patients regardless of test.

In [4]:
def generate_patient_A(p, s, pmf_t):
    while True:
        t = pmf_t.Random()
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        return test, sick
                
outcomes = run_single_simulation(generate_patient_A)
outcomes.Print()

False 0.747960440878
True 0.252039559122


Scenario B:  Choose `t` before generating patients, yield all patients regardless of test.

In [5]:
def generate_patient_B(p, s, pmf_t):
    t = pmf_t.Random()
    while True:
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        return test, sick
                
outcomes = run_single_simulation(generate_patient_B)
outcomes.Print()

False 0.750424193616
True 0.249575806384


Scenario C:  Choose `t` for each patient, only yield patients who test positive.

In [6]:
def generate_patient_C(p, s, pmf_t):
    while True:
        t = pmf_t.Random()
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        if test:
            return test, sick
                
outcomes = run_single_simulation(generate_patient_C)
outcomes.Print()

False 0.75002
True 0.24998


Scenario D:  Choose `t` before generating patients, only yield patients who test positive.

In [7]:
def generate_patient_D(p, s, pmf_t):
    t = pmf_t.Random()
    while True:
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        if test:
            return test, sick
                
outcomes = run_single_simulation(generate_patient_D)
outcomes.Print()

False 0.733227
True 0.266773


Here's a version of the Scenario D simulation optimized for readability.

In [8]:
from random import choice

p = 0.1
s = 0.9
num_sick = 0

for i in range(10000000):
    # decide what the value of t is
    t = choice([0.2, 0.4])
        
    # generate patients until we get a positive test
    while True:
        sick = flip(p)
        test = flip(s) if sick else flip(t)
        if test:
            # count the number of positive tests that are sick
            if sick:
                num_sick += 1

            # break out of the inner loop and re-choose t
            break

In [9]:
num_sick

2665791